# Transfer Learning

In the image transfer learning example, we use a pre-trained Inception_V1 model as image feature transformer and train another linear classifier to solve the dogs-vs-cats classification problem.

In this notebook we are going to take a slightly different approach. We will still use a pre-trained Inception_V1 model, but this time we will operate on the pre-trained model to freeze first of a few layers, replace the classifier on the top, then fine tune the whole model.

# Preparation

## Get the dogs-vs-cats datasets

Download the training dataset from https://www.kaggle.com/c/dogs-vs-cats and extract it. 

The following commands copy about 1100 images of cats and dogs into demo/cats and demo/dogs separately. 
```shell
mkdir -p demo/dogs
mkdir -p demo/cats
cp train/cat.7* demo/cats
cp train/dog.7* demo/dogs
```

## Get the pre-trained Inception-V1 model

Download the pre-trained Inception-V1 model from [Zoo](https://s3-ap-southeast-1.amazonaws.com/bigdl-models/imageclassification/imagenet/bigdl_inception-v1_imagenet_0.4.0.model) 
 Alternatively, user may also download pre-trained caffe/Tensorflow/keras model. Please refer to programming guide in  [BigDL](https://bigdl-project.github.io/)

In [1]:
import re

from bigdl.nn.criterion import CrossEntropyCriterion
from pyspark import SparkConf
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType

from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Dense, Input, Flatten
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *

Adding /home/arda/chentao/MyPython/lib/python2.7/site-packages/bigdl/share/lib/bigdl-0.5.0-jar-with-dependencies.jar to BIGDL_JARS
Prepending /home/arda/chentao/MyPython/lib/python2.7/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
creating: createDefault
creating: createSGD
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor


In [2]:
sparkConf = SparkConf().setAppName("ImageTransferLearningExample")
sc = init_nncontext(sparkConf)

In [3]:
model_path = "/home/arda/chentao/work/xiaxue/bigdl_inception-v1_imagenet_0.4.0.model"
image_path = "/home/arda/chentao/work/xiaxue/demo/*/*"
imageDF = NNImageReader.readImages(image_path, sc)

In [4]:
getName = udf(lambda row:
                  re.search(r'(cat|dog)\.([\d]*)\.jpg', row[0], re.IGNORECASE).group(0),
                  StringType())
getLabel = udf(lambda name: 1.0 if name.startswith('cat') else 2.0, DoubleType())

labelDF = imageDF.withColumn("name", getName(col("image"))) \
        .withColumn("label", getLabel(col('name')))
(trainingDF, validationDF) = labelDF.randomSplit([0.9, 0.1])

compose a pipeline that includes feature transform, pretrained model and Logistic Regression

In [5]:
transformer = ChainedPreprocessing(
        [RowToImageFeature(), ImageResize(256, 256), ImageCenterCrop(224, 224),
         ImageChannelNormalize(123.0, 117.0, 104.0), ImageMatToTensor(), ImageFeatureToTensor()])

creating: createRowToImageFeature
creating: createImageResize
creating: createImageCenterCrop
creating: createImageChannelNormalize
creating: createImageMatToTensor
creating: createImageFeatureToTensor
creating: createChainedPreprocessing


In [6]:
full_model = Net.load_bigdl(model_path)
# create a new model by remove layers after pool5/drop_7x7_s1
model = full_model.new_graph(["pool5/drop_7x7_s1"])
# freeze layers from input to pool4/3x3_s2 inclusive
model.freeze_up_to(["pool4/3x3_s2"])

In [7]:
inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(2)(flatten)

creating: createZooKerasInput
creating: createZooKerasFlatten
creating: createZooKerasDense


In [8]:
lrModel = Model(inputNode, logits)

creating: createZooKerasModel


In [9]:
classifier = NNClassifier(lrModel, CrossEntropyCriterion(), transformer) \
        .setLearningRate(0.003).setBatchSize(40).setMaxEpoch(1).setFeaturesCol("image") \
        .setCachingSample(False)

creating: createCrossEntropyCriterion
creating: createScalarToTensor
creating: createFeatureLabelPreprocessing
creating: createNNClassifier


In [10]:
pipeline = Pipeline(stages=[classifier])

In [11]:
catdogModel = pipeline.fit(trainingDF)
predictionDF = catdogModel.transform(validationDF).cache()

creating: createToTuple
creating: createChainedPreprocessing
creating: createTensorToSample
creating: createChainedPreprocessing


In [12]:
correct = predictionDF.filter("label=prediction").count()
overall = predictionDF.count()
accuracy = correct * 1.0 / overall

In [13]:
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.0372093 
